### Задание  
Взять тот же датасет и предобученную модель для задачи суммаризации  
1. Проверить насколько хорошо она суммаризирует  
2. Сделать генерацию заголовков для статьи

In [1]:
import json
import torch
from transformers import MBartTokenizer, MBartForConditionalGeneration
from datasets import load_dataset

In [2]:
gazeta_test = load_dataset('IlyaGusev/gazeta', revision="v1.0")["test"]

No config specified, defaulting to: gazeta/default
Reusing dataset gazeta (C:\Users\User\.cache\huggingface\datasets\IlyaGusev___gazeta\default\1.0.0\ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
gazeta_test

Dataset({
    features: ['text', 'summary', 'title', 'date', 'url'],
    num_rows: 5770
})

In [4]:
def gen_batch(inputs, batch_size):
    batch_start = 0
    while batch_start < len(inputs):
        yield inputs[batch_start: batch_start + batch_size]
        batch_start += batch_size

In [5]:
def predict(
    model_name,
    input_records,
    target_field,
    max_source_tokens_count=600,
    max_target_tokens_count=160,
    batch_size=4,
    batch_count=5
):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    tokenizer = MBartTokenizer.from_pretrained(model_name)
    model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)
    
    predictions = []
    targets = []
    for num, batch in enumerate(gen_batch(input_records, batch_size), 1):
        if num > batch_count:
            break
        texts = [r['text'] for r in batch]
        target = [r[target_field] for r in batch]
        input_ids = tokenizer(
            texts,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=max_source_tokens_count
        )["input_ids"].to(device)
        
        output_ids = model.generate(
            input_ids=input_ids,
            max_length=max_target_tokens_count + 2,
            no_repeat_ngram_size=3,
            num_beams=5,
            top_k=0
        )
        summaries = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        predictions.extend(summaries)
        targets.extend(target)

    return predictions, targets

In [6]:
predictions, targets = predict("IlyaGusev/mbart_ru_sum_gazeta",
                               list(gazeta_test),
                               'summary')

In [7]:
for i in range(len(predictions)):
    print(f'----------------------------------------------------')
    print(f'Target:\n {targets[i]}')
    print(f'Predictions:\n {predictions[i]}')

----------------------------------------------------
Target:
 В NASA назвали четыре миссии в дальний космос, которые в этом десятилетии могут быть запущены американцами. Среди них — две миссии по изучению Венеры, полет к спутнику Юпитера и экспедиция к Тритону, спутнику Нептуна.
Predictions:
 Американское аэрокосмическое агентство NASA огласило названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации и запуск которых может состояться уже в конце этого десятилетия. Все они были отобраны по критериям потенциальной пользы для науки и технической осуществимости.
----------------------------------------------------
Target:
 25 и 26 февраля в Кремлевском дворце съездов праздновали Сагаалган — Восточный Новый год. Бурятия - центр российского буддизма и один из немногих регионов страны, где новый год встречают официально дважды.
Predictions:
 В Кремле прошла премьера новогоднего шоу «Танцуют все!» с участием более 300 артистов из одного региона. Зрителям 

In [8]:
predictions_title, targets_title = predict("IlyaGusev/mbart_ru_sum_gazeta",
                                           list(gazeta_test),
                                           'title',
                                           max_target_tokens_count=10)

In [9]:
for i in range(len(predictions_title)):
    print(f'----------------------------------------------------')
    print(f'Target:\n {targets_title[i]}')
    print(f'Predictions:\n {predictions_title[i]}')

----------------------------------------------------
Target:
 Венера, Ио или Тритон: куда полетит NASA
Predictions:
 Американское аэрокосмическое агентство NASA объявило
----------------------------------------------------
Target:
 «Люди в Бурятии очень талантливые»
Predictions:
 На главной сцене Кремлевского дворца прошло
----------------------------------------------------
Target:
 Вспомнить СССР: как Лукашенко провел выборы
Predictions:
 Президент Белоруссии Александр Лукашенко назначил
----------------------------------------------------
Target:
 «Он очень переживал»: Бабкина об отношениях с молодым мужем
Predictions:
 Народная артистка РСФСР Надежда Бабкина
----------------------------------------------------
Target:
 «Поддерживают Россию»: почему Киев не платит пенсии Донбассу
Predictions:
 Депутат Верховной рады Елизавета Богу
----------------------------------------------------
Target:
 «Новый кулак в Арктике»: в Дании испугались «Ивана Папанина»
Predictions:
 В Дании выразили 

__ВЫВОДЫ:__ Модель показывает довольно неплохое качество решения задачи саммаризации. Добиться хороших результатов в генерации заголовков посредством параметра max_target_tokens_count не удалось.